# Program 8 - Bayesian Network

We have used the pgmpy library for:

- Maximum Likelihood Estimator (from `pgmpy.estimators`)
- Bayesian Model  (from `pgmpy.models`)
- Variable Elimination (from `pgmpy.inference`)

In [1]:
import numpy as np
import pandas as pd
import csv

from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination

Read the features from the heart disease csv file.

Attributes are given in a python list below

In [2]:
# Read attributes
attributes = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','heartdisease']

# Read features, with the names stored in the variable 'attributes' above 
heartdisease = pd.read_csv('prog7_dataset_heartdisease.csv', names = attributes)
heartdisease = heartdisease.replace('?', np.nan)

Display some of the data stored in the Heart Disese data set

In [5]:
print(heartdisease.head())

    age  sex   cp  trestbps   chol  fbs  restecg  thalach  exang  oldpeak  slope   ca thal  heartdisease
0  63.0  1.0  1.0     145.0  233.0  1.0      2.0    150.0    0.0      2.3    3.0  0.0  6.0             0
1  67.0  1.0  4.0     160.0  286.0  0.0      2.0    108.0    1.0      1.5    2.0  3.0  3.0             2
2  67.0  1.0  4.0     120.0  229.0  0.0      2.0    129.0    1.0      2.6    2.0  2.0  7.0             1
3  37.0  1.0  3.0     130.0  250.0  0.0      0.0    187.0    0.0      3.5    3.0  0.0  3.0             0
4  41.0  0.0  2.0     130.0  204.0  0.0      2.0    172.0    0.0      1.4    1.0  0.0  3.0             0


Some Attributes and Datatypes

In [6]:
print(heartdisease.dtypes)

age             float64
sex             float64
cp              float64
trestbps        float64
chol            float64
fbs             float64
restecg         float64
thalach         float64
exang           float64
oldpeak         float64
slope           float64
ca               object
thal             object
heartdisease      int64
dtype: object


## Create Bayesian Model

Here we create our Bayesian model using the `pgmpy.models.BayesianModel` class.

The input to the `BayesianModel` class is an edge list for the Bayeseian Model graph

---
Examples:
Create an empty bayesian model with no nodes and no edges.
    
    from pgmpy.models import BayesianModel
    G = BayesianModel()
    # a list of edges,
    G.add_edges_from([('a', 'b'), ('b', 'c')])

In [7]:
model = BayesianModel([
    ('age', 'trestbps'),
    ('age', 'fbs'),
    ('sex', 'trestbps'),
    ('sex', 'trestbps'),
    ('exang', 'trestbps'),
    ('trestbps','heartdisease'),
    ('fbs','heartdisease'),
    ('heartdisease','restecg'),
    ('heartdisease','thalach'),
    ('heartdisease','chol')
])

Here, we are learning the CPDs using Maximum Likelihood Estimators

In [8]:
model.fit(heartdisease, estimator = MaximumLikelihoodEstimator)

Inferencing with the Bayesian Network (from Variable Elimination)

In [9]:
heartdisease_infer = VariableElimination(model)

Computing probability of Heart disease given age (age = 28)

In [12]:
q = heartdisease_infer.query(variables = ['heartdisease'], evidence = {'age': 28})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.6791 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.1212 │
├────────────────┼─────────────────────┤
│ heartdisease_2 │              0.0810 │
├────────────────┼─────────────────────┤
│ heartdisease_3 │              0.0939 │
├────────────────┼─────────────────────┤
│ heartdisease_4 │              0.0247 │
╘════════════════╧═════════════════════╛


Computing probability of Heart disease given cholesterol (chol = 100)

In [13]:
q = heartdisease_infer.query(variables = ['heartdisease'], evidence = {'chol': 100})
print(q['heartdisease'])

╒════════════════╤═════════════════════╕
│ heartdisease   │   phi(heartdisease) │
╞════════════════╪═════════════════════╡
│ heartdisease_0 │              0.5400 │
├────────────────┼─────────────────────┤
│ heartdisease_1 │              0.1533 │
├────────────────┼─────────────────────┤
│ heartdisease_2 │              0.1303 │
├────────────────┼─────────────────────┤
│ heartdisease_3 │              0.1259 │
├────────────────┼─────────────────────┤
│ heartdisease_4 │              0.0506 │
╘════════════════╧═════════════════════╛
